In [1]:
import simpy
import numpy as np

# Parámetros globales
N = 3  # Número de ONUs
R = 1_000  # Tasa total de transmisión (Mbps)
T_cycle = 2  # Longitud del ciclo de polling (ms)
T_guard = 0.1  # Tiempo de resguardo (ms)
W = [0.4, 0.3, 0.3]  # SLA de cada ONU (deben sumar 1)

# Clase ONU
class ONU:
    def __init__(self, id, avg_rate, sla_weight):
        self.id = id
        self.avg_rate = avg_rate  # Tasa promedio de generación (paquetes/ms)
        self.sla_weight = sla_weight  # SLA asociado a la ONU
        self.ef_request = 0  # Solicitudes para EF
        self.af_request = 0  # Solicitudes para AF
        self.be_request = 0  # Solicitudes para BE
        self.ef_grant = 0  # Asignación EF
        self.af_grant = 0  # Asignación AF
        self.be_grant = 0  # Asignación BE
        self.b_min = 0  # Ancho de banda mínimo garantizado (B_min)
        self.b_prime = 0  # Ancho de banda restante (B')

    def generate_requests(self):
        """Generar solicitudes acumuladas para el ciclo."""
        self.ef_request = np.random.poisson(self.avg_rate * 0.2)  # 20% EF
        self.af_request = np.random.poisson(self.avg_rate * 0.5)  # 50% AF
        self.be_request = np.random.poisson(self.avg_rate * 0.3)  # 30% BE

# Clase OLT
class OLT:
    def __init__(self, env, num_onus, cycle_time, total_bandwidth, guard_time, sla_weights):
        self.env = env
        self.num_onus = num_onus
        self.cycle_time = cycle_time
        self.total_bandwidth = total_bandwidth
        self.guard_time = guard_time
        self.sla_weights = sla_weights
        self.onus = []  # Lista de ONUs

    def calculate_min_bandwidth(self):
        """Calcular B_min para cada ONU."""
        denominator = 8 * sum(self.sla_weights)
        for onu in self.onus:
            onu.b_min = (
                (self.cycle_time - 2 * self.num_onus * self.guard_time)
                * onu.sla_weight
                * self.total_bandwidth
            ) / denominator

    def allocate_bandwidth(self):
        """Asignar ancho de banda basado en prioridades."""
        ef_total = sum(onu.ef_request for onu in self.onus)
        af_be_total = sum(onu.af_request + onu.be_request for onu in self.onus)

        # Asignación EF
        for onu in self.onus:
            if ef_total > 0:
                onu.ef_grant = (onu.ef_request / ef_total) * self.total_bandwidth * 0.6  # 60% para EF
            else:
                onu.ef_grant = 0

        # Calcular B' después de la asignación de EF
        for onu in self.onus:
            onu.b_prime = max(onu.b_min - onu.ef_grant, 0)  # B' no puede ser negativo

        # Asignación AF y BE (40% restante)
        remaining_bw = self.total_bandwidth - sum(onu.ef_grant for onu in self.onus)
        for onu in self.onus:
            if af_be_total > 0:
                onu.af_grant = (onu.af_request / af_be_total) * remaining_bw * 0.7  # 70% para AF
                onu.be_grant = (onu.be_request / af_be_total) * remaining_bw * 0.3  # 30% para BE
            else:
                onu.af_grant, onu.be_grant = 0, 0

    def run(self):
        """Ejecutar ciclos de polling."""
        while True:
            # Calcular B_min para cada ONU
            self.calculate_min_bandwidth()

            # Recolectar solicitudes y asignar ancho de banda
            for onu in self.onus:
                onu.generate_requests()
            self.allocate_bandwidth()

            # Imprimir resultados del ciclo
            print(f"Cycle {self.env.now}:")
            for onu in self.onus:
                print(f"ONU {onu.id}: B_min={onu.b_min:.2f}, EF={onu.ef_request}/{onu.ef_grant:.2f}, "
                      f"AF={onu.af_request}/{onu.af_grant:.2f}, BE={onu.be_request}/{onu.be_grant:.2f}, "
                      f"B'={onu.b_prime:.2f}")
            yield self.env.timeout(self.cycle_time)

# Crear entorno
env = simpy.Environment()
olt = OLT(env, N, T_cycle, R, T_guard, W)

# Crear ONUs y agregarlas al OLT
for i in range(N):
    onu = ONU(i, avg_rate=100, sla_weight=W[i])  # Tasa promedio de generación (paquetes/ms)
    olt.onus.append(onu)

# Iniciar simulación
env.process(olt.run())
env.run(until=10)  # Simular 10 ms

Cycle 0:
ONU 0: B_min=700.00, EF=20/1690.14, AF=51/568.92, BE=31/148.21, B'=0.00
ONU 1: B_min=525.00, EF=22/1859.15, AF=49/546.61, BE=34/162.55, B'=0.00
ONU 2: B_min=525.00, EF=29/2450.70, AF=51/568.92, BE=35/167.33, B'=0.00
Cycle 2:
ONU 0: B_min=700.00, EF=24/2322.58, AF=49/571.67, BE=39/195.00, B'=0.00
ONU 1: B_min=525.00, EF=22/2129.03, AF=50/583.33, BE=25/125.00, B'=0.00
ONU 2: B_min=525.00, EF=16/1548.39, AF=39/455.00, BE=38/190.00, B'=0.00
Cycle 4:
ONU 0: B_min=700.00, EF=20/1967.21, AF=55/616.00, BE=22/105.60, B'=0.00
ONU 1: B_min=525.00, EF=19/1868.85, AF=44/492.80, BE=40/192.00, B'=0.00
ONU 2: B_min=525.00, EF=22/2163.93, AF=57/638.40, BE=32/153.60, B'=0.00
Cycle 6:
ONU 0: B_min=700.00, EF=16/1745.45, AF=61/683.20, BE=34/163.20, B'=0.00
ONU 1: B_min=525.00, EF=18/1963.64, AF=45/504.00, BE=34/163.20, B'=0.00
ONU 2: B_min=525.00, EF=21/2290.91, AF=44/492.80, BE=32/153.60, B'=0.00
Cycle 8:
ONU 0: B_min=700.00, EF=26/2228.57, AF=51/610.26, BE=29/148.72, B'=0.00
ONU 1: B_min=525.00